In [1]:
# Import modules
import sys
# If your authentification script is not in the project directory
# append its folder to sys.path
# sys.path.append("../spotify_api_web_app")
# import authorization
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from decouple import config

In [2]:
df = pd.read_csv("music_characteristics_dataset3.csv")

In [3]:
df["mood_vec"] = df[["valence", "energy", "acousticness", "instrumentalness", "speechiness", "mode"]].values.tolist()
df

,id,genre,track_name,artist_name,popularity,valence,energy,danceability,acousticness,tempo,speechiness,mode,instrumentalness,mood_vec
0,7GILkDXz9y9x5KPHBKdrHA,acoustic,Spring för livet,Sara Varga,0,0.5980,0.572,0.786,0.2930,119.943,0.0590,1,0.000000,"[0.598, 0.5720000000000001, 0.293, 0.0, 0.0590..."
1,2xwsf9FuFINP1X4FTsqZ7Q,acoustic,Peace,O.A.R.,62,0.4930,0.721,0.449,0.1660,178.141,0.0392,1,0.000000,"[0.493, 0.721, 0.166, 0.0, 0.0392, 1.0]"
2,4VJgrWjrkodaGiq3xKz62z,acoustic,Sometimes (I Wish),City and Colour,40,0.1350,0.467,0.523,0.3110,129.953,0.0281,0,0.000020,"[0.135, 0.467, 0.311, 2.01e-05, 0.0281, 0.0]"
3,46XMysg4VurmyAQ28tshqz,acoustic,The Mess I Made,Parachute,47,0.1730,0.591,0.405,0.0153,165.377,0.0339,1,0.000000,"[0.17300000000000001, 0.591, 0.0153, 0.0, 0.03..."
4,25J4d78ESH2MgAJoIakB8O,acoustic,Lost In The Light,Bahamas,50,0.2460,0.267,0.597,0.7930,75.535,0.0309,1,0.001400,"[0.24600000000000002, 0.267, 0.792999999999999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,3CCGsOj2efotChiScMmCWg,world-music,A Minha Menina,Os Mutantes,0,0.4290,0.930,0.455,0.0568,91.410,0.1260,1,0.000000,"[0.429, 0.93, 0.0568, 0.0, 0.126, 1.0]"
3001,6bYpEZmbTldBmQtnnfDRDk,world-music,Raoui,Various Artists,0,0.6250,0.058,0.560,0.9260,107.658,0.0500,0,0.000003,"[0.625, 0.057999999999999996, 0.92599999999999..."
3002,1xJtdt697VaDWFJ1eEwlPO,world-music,Cat Bed Music,Pet Music World,5,0.0394,0.033,0.304,0.9920,71.282,0.0655,1,0.963000,"[0.0394, 0.033, 0.992, 0.963, 0.0655, 1.0]"
3003,1Nov1m4HA8PBs2vyWjp2Cv,world-music,Mas Que Nada,Jorge Ben Jor,0,0.5980,0.403,0.514,0.6720,89.672,0.0325,0,0.000366,"[0.598, 0.40299999999999997, 0.672, 0.000366, ..."


In [7]:
# CREATE CLIENT FUNCTION TO CONNECT US TO SPOTIFY
def client():
    client_id = config('SPOTIPY_CLIENT_ID')
    client_secret = config('SPOTIPY_CLIENT_SECRET')
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    return spotify

In [11]:
# GET TRACK ID

# 1. Get User input
# mood_sample_track = input("Sample Track:")
# sample_artist = input("Sample Artist:")

# Makes list to hold track_id. Also artist_name and track_name to verify that it is the correct track
artist_name = []
track_name = []
track_id = []

# Testing sample
sample_track = 'calypso'
sample_artist = 'john denver'

# client()
client_id = config('SPOTIPY_CLIENT_ID')
client_secret = config('SPOTIPY_CLIENT_SECRET')
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

track_result = spotify.search(q=f'track:{sample_track} artist:{sample_artist}', limit=1, type='track')
    
for i, t in enumerate(track_result['tracks']['items']):
    artist_name.append(t['artists'][0]['name'])
    track_name.append(t['name'])
    track_id.append(t['id'])

track_id = track_id[0]

In [18]:
# CREATE RECOMMENDATION FUNCTION
def recommend(id, ref_df, n_recs = 5):
    
    # Crawl valence and arousal of given track from spotify api
    track_features = spotify.audio_features(track_id)[0]
    track_moodvec = np.array([track_features['valence'], track_features['energy'], track_features['acousticness'], 
                              track_features['instrumentalness'], track_features['speechiness'], 
                              track_features['mode']])
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: np.linalg.norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True) 
    ref_df_sorted = ref_df.sort_values(by = "popularity", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommended
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

In [19]:
recommend(id = track_id, ref_df = df, n_recs = 5)

,id,genre,track_name,artist_name,popularity,valence,energy,danceability,acousticness,tempo,speechiness,mode,instrumentalness,mood_vec,distances
0,7GILkDXz9y9x5KPHBKdrHA,acoustic,Spring för livet,Sara Varga,0,0.598,0.572,0.786,0.293000,119.943,0.0590,1,0.000000,"[0.598, 0.5720000000000001, 0.293, 0.0, 0.0590...",0.556140
943,3nqP8YOhxJPdUzXJWbdKyT,funk,Don't Stop (feat. Tiffany Austin),Orgone,0,0.928,0.880,0.778,0.080300,111.944,0.0324,0,0.000141,"[0.9279999999999999, 0.88, 0.0803, 0.000140999...",1.391814
2053,3IvVQ1qmekCQ4LRwGCyIJA,party,PMW (All I Really Need) (feat. ScHoolboy Q),A$AP Rocky,0,0.269,0.752,0.697,0.000367,130.022,0.0526,1,0.000000,"[0.26899999999999996, 0.752, 0.000367, 0.0, 0....",0.909411
941,06ObEGGlc8iwNI3vmtRm1s,funk,Standing On The Verge Of Getting It On,Funkadelic,0,0.757,0.777,0.747,0.058200,116.142,0.1230,1,0.000000,"[0.757, 0.777, 0.0582, 0.0, 0.1230000000000000...",0.881972
940,3hPjqX0kd294vzmyKvJn6u,funk,I Couldn't Love You More,Incognito,0,0.619,0.688,0.572,0.097300,93.004,0.0698,1,0.000035,"[0.619, 0.688, 0.0973, 3.48e-05, 0.0698, 1.0]",0.778298
